# 3 Pointers Made against game_details.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis\3p
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [5]:
df = pd.read_csv('backend/data/details/game_details.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = shooting_df[['date', 'visitor', 'home', 'team', '3p']]

### Basic exploration

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504089 entries, 0 to 504088
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        504089 non-null  object 
 1   visitor     504089 non-null  object 
 2   home        504089 non-null  object 
 3   team        504089 non-null  int64  
 4   starter     504047 non-null  float64
 5   player      503719 non-null  object 
 6   mp          503719 non-null  object 
 7   fg          503719 non-null  float64
 8   fga         503719 non-null  float64
 9   fg_perc     482352 non-null  float64
 10  3p          503719 non-null  float64
 11  3pa         503719 non-null  float64
 12  3p_perc     353016 non-null  float64
 13  ft          503719 non-null  float64
 14  fta         503719 non-null  float64
 15  ft_perc     326899 non-null  float64
 16  orb         503719 non-null  float64
 17  drb         503719 non-null  float64
 18  trb         503719 non-null  float64
 19  as

In [7]:
df.sample(5)

,date,visitor,home,team,starter,player,mp,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus
294111,"Wed, Dec 9, 2015",Orlando Magic,Phoenix Suns,0,0.0,Jason Smith,6:48,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0
44240,"Sat, Jan 12, 2008",Minnesota Timberwolves,San Antonio Spurs,0,0.0,Corey Brewer,16:47,1.0,4.0,0.250,0.0,0.0,NaN,0.0,0.0,NaN,1.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,-6.0
87887,"Sun, Mar 22, 2009",Oklahoma City Thunder,Minnesota Timberwolves,0,0.0,Earl Watson,0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251450,"Fri, May 9, 2014",Oklahoma City Thunder,Los Angeles Clippers,1,0.0,Jared Dudley,7:54,1.0,3.0,0.333,1.0,2.0,0.5,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,-4.0
376741,"Sun, Mar 4, 2018",Charlotte Hornets,Toronto Raptors,1,0.0,Delon Wright,13:04,0.0,1.0,0.000,0.0,0.0,NaN,4.0,6.0,0.667,0.0,1.0,1.0,4.0,0.0,0.0,0.0,1.0,4.0,-8.0


In [8]:
def convert_mp(mp):
    if mp == '0' or mp == 0:
        return 0
    else:
        mins = int(mp.split(':')[0])
        secs = int(mp.split(':')[1]) / 60
        return mins + secs

In [9]:
# Fill NaN
df = df.fillna(0)

# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Convert 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Convert 'minutes played' to float
df['mp'] = df['mp'].apply(lambda x: convert_mp(x))

# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504089 entries, 0 to 504088
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        504089 non-null  datetime64[ns]
 1   visitor     504089 non-null  object        
 2   home        504089 non-null  object        
 3   team        504089 non-null  object        
 4   starter     504089 non-null  float64       
 5   player      504089 non-null  object        
 6   mp          504089 non-null  float64       
 7   fg          504089 non-null  float64       
 8   fga         504089 non-null  float64       
 9   fg_perc     504089 non-null  float64       
 10  3p          504089 non-null  float64       
 11  3pa         504089 non-null  float64       
 12  3p_perc     504089 non-null  float64       
 13  ft          504089 non-null  float64       
 14  fta         504089 non-null  float64       
 15  ft_perc     504089 non-null  float64       
 16  or

In [10]:
# Team total stats
teams_df = df.groupby(['date', 'visitor', 'home', 'team']).sum().reset_index()

In [11]:
# Rename target variable
shooting_df = shooting_df.rename({'3p': 'target'}, axis=1)

# Convert 'date' column to Date object
shooting_df['date'] = pd.to_datetime(shooting_df['date'])

# Convert 'team' column to Team Name
shooting_df['team'] = np.where(shooting_df['team'], shooting_df['home'], shooting_df['visitor'])

In [40]:
# Starters total stats
starters_df = df[df['starter'] == 1].groupby(['date', 'visitor', 'home', 'team']).sum()
starters_df = starters_df[stats]
starters_df = starters_df.reset_index()

# Merge dataframes to have target variable
starters_df = pd.merge(starters_df, shooting_df, 
                       left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
                       how='left')

In [41]:
# Bench total stats
bench_df = df[df['starter'] == 0].groupby(['date', 'visitor', 'home', 'team']).sum()
bench_df = bench_df[stats]
bench_df = bench_df.reset_index()

# Merge dataframes to have target variable
bench_df = pd.merge(bench_df, shooting_df, 
                    left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
                    how='left')

# Dataframe of team's last 15 performances

In [16]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = teams_df[teams_df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

teams_df['dates'] = teams_df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
teams_df['date_1'], teams_df['date_2'] = teams_df['dates'].apply(lambda x: x[0]), teams_df['dates'].apply(lambda x: x[1])
teams_df['date_3'], teams_df['date_4'] = teams_df['dates'].apply(lambda x: x[2]), teams_df['dates'].apply(lambda x: x[3])
teams_df['date_5'], teams_df['date_6'] = teams_df['dates'].apply(lambda x: x[4]), teams_df['dates'].apply(lambda x: x[5])
teams_df['date_7'], teams_df['date_8'] = teams_df['dates'].apply(lambda x: x[6]), teams_df['dates'].apply(lambda x: x[7])
teams_df['date_9'], teams_df['date_10'] = teams_df['dates'].apply(lambda x: x[8]), teams_df['dates'].apply(lambda x: x[9])
teams_df['date_11'], teams_df['date_12'] = teams_df['dates'].apply(lambda x: x[10]), teams_df['dates'].apply(lambda x: x[11])
teams_df['date_13'], teams_df['date_14'] = teams_df['dates'].apply(lambda x: x[12]), teams_df['dates'].apply(lambda x: x[13])
teams_df['date_15'] = teams_df['dates'].apply(lambda x: x[14])

In [42]:
# Merge in opponents (see team defensive stats)
starters_df = pd.merge(
    starters_df, 
    starters_df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

starters_df = starters_df[starters_df['team'] != starters_df['team_opp']]

bench_df = pd.merge(
    bench_df, 
    bench_df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

bench_df = bench_df[bench_df['team'] != bench_df['team_opp']]

In [43]:
# Keep date columns in teams
cols = [col for col in teams_df.columns
        if ('date_' in col) or \
        (col in ['date', 'visitor', 'home', 'team'])]
teams_df = teams_df[cols]

# Merge dates with starters
starters_df = pd.merge(starters_df, teams_df, 
                       left_on=['date', 'visitor', 'home', 'team'], 
                       right_on=['date', 'visitor', 'home', 'team'],
                       how='left')

# Merge dates with bench
bench_df = pd.merge(bench_df, teams_df, 
                    left_on=['date', 'visitor', 'home', 'team'], 
                    right_on=['date', 'visitor', 'home', 'team'],
                    how='left')

In [44]:
starters_df.head()

,date,visitor,home,team,fg,fga,3p,3pa,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,mp,target,team_opp,fg_opp,fga_opp,3p_opp,3pa_opp,ft_opp,fta_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,plus_minus_opp,mp_opp,target_opp,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15
0,2006-10-31,Chicago Bulls,Miami Heat,Chicago Bulls,18.0,43.0,3.0,7.0,14.0,18.0,7.0,14.0,21.0,8.0,5.0,2.0,7.0,12.0,53.0,66.0,128.550000,7.0,Miami Heat,20.0,48.0,3.0,13.0,11.0,19.0,4.0,18.0,22.0,9.0,4.0,3.0,14.0,14.0,54.0,-95.0,156.583333,3.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2006-10-31,Chicago Bulls,Miami Heat,Miami Heat,20.0,48.0,3.0,13.0,11.0,19.0,4.0,18.0,22.0,9.0,4.0,3.0,14.0,14.0,54.0,-95.0,156.583333,3.0,Chicago Bulls,18.0,43.0,3.0,7.0,14.0,18.0,7.0,14.0,21.0,8.0,5.0,2.0,7.0,12.0,53.0,66.0,128.550000,7.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2006-10-31,Phoenix Suns,Los Angeles Lakers,Los Angeles Lakers,26.0,51.0,5.0,10.0,15.0,20.0,7.0,26.0,33.0,24.0,7.0,0.0,13.0,10.0,72.0,-5.0,157.600000,6.0,Phoenix Suns,24.0,49.0,6.0,18.0,5.0,5.0,4.0,20.0,24.0,22.0,1.0,4.0,11.0,13.0,59.0,-11.0,159.483333,13.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2006-10-31,Phoenix Suns,Los Angeles Lakers,Phoenix Suns,24.0,49.0,6.0,18.0,5.0,5.0,4.0,20.0,24.0,22.0,1.0,4.0,11.0,13.0,59.0,-11.0,159.483333,13.0,Los Angeles Lakers,26.0,51.0,5.0,10.0,15.0,20.0,7.0,26.0,33.0,24.0,7.0,0.0,13.0,10.0,72.0,-5.0,157.600000,6.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,2006-11-01,Atlanta Hawks,Philadelphia 76ers,Atlanta Hawks,15.0,49.0,3.0,12.0,15.0,18.0,8.0,19.0,27.0,7.0,7.0,4.0,15.0,16.0,48.0,-83.0,164.683333,4.0,Philadelphia 76ers,24.0,56.0,3.0,5.0,21.0,23.0,13.0,21.0,34.0,17.0,7.0,5.0,14.0,12.0,72.0,78.0,168.800000,3.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [18]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

# Starters Analysis

In [45]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = starters_df.copy()
X = starters_df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

last_15_games.head()

,date,visitor,home,team,fg,fga,3p,3pa,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,mp,target,team_opp,fg_opp,fga_opp,3p_opp,3pa_opp,ft_opp,fta_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,plus_minus_opp,mp_opp,target_opp,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15,date_1,fg_1,fga_1,3p_1,3pa_1,ft_1,fta_1,orb_1,drb_1,trb_1,ast_1,stl_1,blk_1,tov_1,pf_1,pts_1,plus_minus_1,mp_1,fg_opp_1,fga_opp_1,3p_opp_1,3pa_opp_1,ft_opp_1,fta_opp_1,orb_opp_1,drb_opp_1,trb_opp_1,ast_opp_1,stl_opp_1,blk_opp_1,tov_opp_1,pf_opp_1,pts_opp_1,plus_minus_opp_1,mp_opp_1,date_2,fg_2,fga_2,3p_2,3pa_2,ft_2,fta_2,orb_2,drb_2,trb_2,ast_2,stl_2,blk_2,tov_2,pf_2,pts_2,plus_minus_2,mp_2,fg_opp_2,fga_opp_2,3p_opp_2,3pa_opp_2,ft_opp_2,fta_opp_2,orb_opp_2,drb_opp_2,trb_opp_2,ast_opp_2,stl_opp_2,blk_opp_2,tov_opp_2,pf_opp_2,pts_opp_2,plus_minus_opp_2,mp_opp_2,date_3,fg_3,fga_3,3p_3,3pa_3,ft_3,fta_3,orb_3,drb_3,trb_3,ast_3,stl_3,blk_3,tov_3,pf_3,pts_3,plus_minus_3,mp_3,fg_opp_3,fga_opp_3,3p_opp_3,3pa_opp_3,ft_opp_3,fta_opp_3,orb_opp_3,drb_opp_3,trb_opp_3,ast_opp_3,stl_opp_3,blk_opp_3,tov_opp_3,pf_opp_3,pts_opp_3,plus_minus_opp_3,mp_opp_3,date_4,fg_4,fga_4,3p_4,3pa_4,ft_4,fta_4,orb_4,drb_4,trb_4,ast_4,stl_4,blk_4,tov_4,pf_4,pts_4,plus_minus_4,mp_4,fg_opp_4,fga_opp_4,3p_opp_4,3pa_opp_4,ft_opp_4,fta_opp_4,orb_opp_4,drb_opp_4,trb_opp_4,ast_opp_4,stl_opp_4,blk_opp_4,tov_opp_4,pf_opp_4,pts_opp_4,plus_minus_opp_4,mp_opp_4,date_5,fg_5,fga_5,3p_5,3pa_5,ft_5,fta_5,orb_5,drb_5,trb_5,ast_5,stl_5,blk_5,tov_5,pf_5,pts_5,plus_minus_5,mp_5,fg_opp_5,fga_opp_5,3p_opp_5,3pa_opp_5,ft_opp_5,fta_opp_5,orb_opp_5,drb_opp_5,trb_opp_5,ast_opp_5,stl_opp_5,blk_opp_5,tov_opp_5,pf_opp_5,pts_opp_5,plus_minus_opp_5,mp_opp_5,date_6,fg_6,fga_6,3p_6,3pa_6,ft_6,fta_6,orb_6,drb_6,trb_6,ast_6,stl_6,blk_6,tov_6,pf_6,pts_6,plus_minus_6,mp_6,fg_opp_6,fga_opp_6,3p_opp_6,3pa_opp_6,ft_opp_6,fta_opp_6,orb_opp_6,drb_opp_6,trb_opp_6,ast_opp_6,stl_opp_6,blk_opp_6,tov_opp_6,pf_opp_6,pts_opp_6,plus_minus_opp_6,mp_opp_6,date_7,fg_7,fga_7,3p_7,3pa_7,ft_7,fta_7,orb_7,drb_7,trb_7,ast_7,stl_7,blk_7,tov_7,pf_7,pts_7,plus_minus_7,mp_7,fg_opp_7,fga_opp_7,3p_opp_7,3pa_opp_7,ft_opp_7,fta_opp_7,orb_opp_7,drb_opp_7,trb_opp_7,ast_opp_7,stl_opp_7,blk_opp_7,tov_opp_7,pf_opp_7,pts_opp_7,plus_minus_opp_7,mp_opp_7,date_8,fg_8,fga_8,3p_8,3pa_8,ft_8,fta_8,orb_8,drb_8,trb_8,ast_8,stl_8,blk_8,tov_8,pf_8,pts_8,plus_minus_8,mp_8,fg_opp_8,fga_opp_8,3p_opp_8,3pa_opp_8,ft_opp_8,fta_opp_8,orb_opp_8,drb_opp_8,trb_opp_8,ast_opp_8,stl_opp_8,blk_opp_8,tov_opp_8,pf_opp_8,pts_opp_8,plus_minus_opp_8,mp_opp_8,date_9,fg_9,fga_9,3p_9,3pa_9,ft_9,fta_9,orb_9,drb_9,trb_9,ast_9,stl_9,blk_9,tov_9,pf_9,pts_9,plus_minus_9,mp_9,fg_opp_9,fga_opp_9,3p_opp_9,3pa_opp_9,ft_opp_9,fta_opp_9,orb_opp_9,drb_opp_9,trb_opp_9,ast_opp_9,stl_opp_9,blk_opp_9,tov_opp_9,pf_opp_9,pts_opp_9,plus_minus_opp_9,mp_opp_9,date_10,fg_10,fga_10,3p_10,3pa_10,ft_10,fta_10,orb_10,drb_10,trb_10,ast_10,stl_10,blk_10,tov_10,pf_10,pts_10,plus_minus_10,mp_10,fg_opp_10,fga_opp_10,3p_opp_10,3pa_opp_10,ft_opp_10,fta_opp_10,orb_opp_10,drb_opp_10,trb_opp_10,ast_opp_10,stl_opp_10,blk_opp_10,tov_opp_10,pf_opp_10,pts_opp_10,plus_minus_opp_10,mp_opp_10,date_11,fg_11,fga_11,3p_11,3pa_11,ft_11,fta_11,orb_11,drb_11,trb_11,ast_11,stl_11,blk_11,tov_11,pf_11,pts_11,plus_minus_11,mp_11,fg_opp_11,fga_opp_11,3p_opp_11,3pa_opp_11,ft_opp_11,fta_opp_11,orb_opp_11,drb_opp_11,trb_opp_11,ast_opp_11,stl_opp_11,blk_opp_11,tov_opp_11,pf_opp_11,pts_opp_11,plus_minus_opp_11,mp_opp_11,date_12,fg_12,fga_12,3p_12,3pa_12,ft_12,fta_12,orb_12,drb_12,trb_12,ast_12,stl_12,blk_12,tov_12,pf_12,pts_12,plus_minus_12,mp_12,fg_opp_12,fga_opp_12,3p_opp_12,3pa_opp_12,ft_opp_12,fta_opp_12,orb_opp_12,drb_opp_12,trb_opp_12,ast_opp_12,stl_opp_12,blk_opp_12,tov_opp_12,pf_opp_12,pts_opp_12,plus_minus_opp_12,mp_opp_12,date_13,fg_13,fga_13,3p_13,3pa_13,ft_13,fta_13,orb_13,drb_13,trb_13,ast_13,stl_13,blk_13,tov_13,pf_13,pts_13,plus_minus_13,mp_13,

### Last 15 Performances (Unweighted)

In [50]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))]

last_15 = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15[stat + '_trend'] = last_15[stat + '_trend'] + \
                                        z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:10])

last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

# Calculate percentages
last_15['fg_perc'] = last_15['fg'] / last_15['fga']
last_15['fg_perc_opp'] = last_15['fg_opp'] / last_15['fga_opp']

last_15['3p_perc'] = last_15['3p'] / last_15['3pa']
last_15['3p_perc_opp'] = last_15['3p_opp'] / last_15['3pa_opp']

last_15['ft_perc'] = last_15['ft'] / last_15['fta']
last_15['ft_perc_opp'] = last_15['ft_opp'] / last_15['fta_opp']

# Calculate advanced stats
last_15['ts_perc'] = last_15['pts'] / (2 * (last_15['fga'] + .44 * last_15['fta']))
last_15['ts_perc_opp'] = last_15['pts_opp'] / (2 * (last_15['fga_opp'] + .44 * last_15['fta_opp']))

last_15['efg_perc'] = (last_15['fg'] + (.5 * last_15['3p'])) / last_15['fga']
last_15['efg_perc_opp'] = (last_15['fg_opp'] + (.5 * last_15['3p_opp'])) / last_15['fga_opp']

last_15['3par'] = last_15['3pa'] / last_15['fga']
last_15['3par_opp'] = last_15['3pa_opp'] / last_15['fga_opp']

last_15['ftr'] = last_15['fta'] / last_15['fga']
last_15['ftr_opp'] = last_15['fta_opp'] / last_15['fga_opp']

last_15['orb_perc'] = last_15['orb'] / (last_15['orb'] + last_15['drb_opp'])
last_15['orb_perc_opp'] = last_15['orb_opp'] / (last_15['orb_opp'] + last_15['drb'])

last_15['drb_perc'] = last_15['drb'] / (last_15['drb'] + last_15['orb_opp'])
last_15['drb_perc_opp'] = last_15['drb_opp'] / (last_15['drb_opp'] + last_15['orb'])

last_15['trb_perc'] = last_15['trb'] / (last_15['trb'] + last_15['trb_opp'])
last_15['trb_perc_opp'] = last_15['trb_opp'] / (last_15['trb_opp'] + last_15['trb'])

last_15['ast_perc'] = last_15['ast'] / last_15['fg']
last_15['ast_perc_opp'] = last_15['ast_opp'] / last_15['fg_opp']

starters_15_games = last_15.dropna(axis=0).copy()
starters_15_games.tail()

target         fg  \
date       visitor              home                                       
2022-03-06 New Orleans Pelicans Denver Nuggets            0.0  55.466667   
           New York Knicks      Los Angeles Clippers      0.0  53.800000   
           Phoenix Suns         Milwaukee Bucks           0.0  66.400000   
           Toronto Raptors      Cleveland Cavaliers       0.0  55.600000   
           Utah Jazz            Oklahoma City Thunder     0.0  51.933333   

                                                              fga         3p  \
date       visitor              home                                           
2022-03-06 New Orleans Pelicans Denver Nuggets         109.466667  12.266667   
           New York Knicks      Los Angeles Clippers   116.933333  15.866667   
           Phoenix Suns         Milwaukee Bucks        128.733333  17.466667   
           Toronto Raptors      Cleveland Cavaliers    116.533333  14.400000   
           Utah Jazz            Oklahoma City Thunder  113.066667  16.133333   

                                                             3pa         ft  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         38.400000  25.200000   
           New York Knicks      Los Angeles Clippers   42.800000  22.066667   
           Phoenix Suns         Milwaukee Bucks        47.733333  30.000000   
           Toronto Raptors      Cleveland Cavaliers    37.800000  23.466667   
           Utah Jazz            Oklahoma City Thunder  44.733333  22.600000   

                                                             fta        orb  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         30.333333  11.866667   
           New York Knicks      Los Angeles Clippers   30.533333  14.133333   
           Phoenix Suns         Milwaukee Bucks        37.533333  11.066667   
           Toronto Raptors      Cleveland Cavaliers    30.133333  14.400000   
           Utah Jazz            Oklahoma City Thunder  29.266667  13.000000   

                                                             drb        trb  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         46.800000  58.666667   
           New York Knicks      Los Angeles Clippers   43.866667  58.000000   
           Phoenix Suns         Milwaukee Bucks        50.066667  61.133333   
           Toronto Raptors      Cleveland Cavaliers    39.866667  54.266667   
           Utah Jazz            Oklahoma City Thunder  44.533333  57.533333   

                                                             ast        stl  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         36.666667   8.400000   
           New York Knicks      Los Angeles Clippers   29.400000   7.800000   
           Phoenix Suns         Milwaukee Bucks        42.466667  11.133333   
           Toronto Raptors      Cleveland Cavaliers    31.866667  10.066667   
           Utah Jazz            Oklahoma City Thunder  29.466667  10.866667   

                                                            blk        tov  \
date       visitor              home                                         
2022-03-06 New Orleans Pelicans Denver Nuggets         4.733333  19.533333   
           New York Knicks      Los Angeles Clippers   6.000000  16.733333   
           Phoenix Suns         Milwaukee Bucks        4.333333  16.066667   
           Toronto Raptors      Cleveland Cavaliers    5.600000  17.733333   
           Utah Jazz            Oklahoma City Thunder  5.400000  19.333333   

                                                              pf         pts  \
date       visitor              home                                           
2022-03-06 New Orleans Pelicans Denver

## Correlations

In [51]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in starters_15_games:
    corr_p = pearsonr(starters_15_games['target'], starters_15_games[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print statistically significant correlations
starters_corr = corr_df[corr_df['p-value'] < .05].sort_values(['corr'], axis=0, ascending=False)
starters_corr

,corr,p-value,stat
0,1.00,0.0,target
4,0.71,0.0,3pa
113,0.71,0.0,3par
3,0.69,0.0,3p
21,0.68,0.0,3pa_opp
...,...,...,...
115,-0.29,0.0,ftr
24,-0.29,0.0,orb_opp
116,-0.37,0.0,ftr_opp
117,-0.38,0.0,orb_perc


## Bench Analysis

In [52]:
# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = bench_df.copy()
X = bench_df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

### Last 15 Performances (Unweighted)

In [53]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))]

last_15 = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15[stat + '_trend'] = last_15[stat + '_trend'] + \
                                        z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:10])

last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

# Calculate percentages
last_15['fg_perc'] = last_15['fg'] / last_15['fga']
last_15['fg_perc_opp'] = last_15['fg_opp'] / last_15['fga_opp']

last_15['3p_perc'] = last_15['3p'] / last_15['3pa']
last_15['3p_perc_opp'] = last_15['3p_opp'] / last_15['3pa_opp']

last_15['ft_perc'] = last_15['ft'] / last_15['fta']
last_15['ft_perc_opp'] = last_15['ft_opp'] / last_15['fta_opp']

# Calculate advanced stats
last_15['ts_perc'] = last_15['pts'] / (2 * (last_15['fga'] + .44 * last_15['fta']))
last_15['ts_perc_opp'] = last_15['pts_opp'] / (2 * (last_15['fga_opp'] + .44 * last_15['fta_opp']))

last_15['efg_perc'] = (last_15['fg'] + (.5 * last_15['3p'])) / last_15['fga']
last_15['efg_perc_opp'] = (last_15['fg_opp'] + (.5 * last_15['3p_opp'])) / last_15['fga_opp']

last_15['3par'] = last_15['3pa'] / last_15['fga']
last_15['3par_opp'] = last_15['3pa_opp'] / last_15['fga_opp']

last_15['ftr'] = last_15['fta'] / last_15['fga']
last_15['ftr_opp'] = last_15['fta_opp'] / last_15['fga_opp']

last_15['orb_perc'] = last_15['orb'] / (last_15['orb'] + last_15['drb_opp'])
last_15['orb_perc_opp'] = last_15['orb_opp'] / (last_15['orb_opp'] + last_15['drb'])

last_15['drb_perc'] = last_15['drb'] / (last_15['drb'] + last_15['orb_opp'])
last_15['drb_perc_opp'] = last_15['drb_opp'] / (last_15['drb_opp'] + last_15['orb'])

last_15['trb_perc'] = last_15['trb'] / (last_15['trb'] + last_15['trb_opp'])
last_15['trb_perc_opp'] = last_15['trb_opp'] / (last_15['trb_opp'] + last_15['trb'])

last_15['ast_perc'] = last_15['ast'] / last_15['fg']
last_15['ast_perc_opp'] = last_15['ast_opp'] / last_15['fg_opp']

bench_15_games = last_15.dropna(axis=0).copy()
bench_15_games.tail()

target         fg  \
date       visitor              home                                       
2022-03-06 New Orleans Pelicans Denver Nuggets            0.0  29.133333   
           New York Knicks      Los Angeles Clippers      0.0  27.266667   
           Phoenix Suns         Milwaukee Bucks           0.0  20.866667   
           Toronto Raptors      Cleveland Cavaliers       0.0  24.333333   
           Utah Jazz            Oklahoma City Thunder     0.0  28.533333   

                                                             fga         3p  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         62.333333  11.933333   
           New York Knicks      Los Angeles Clippers   60.533333   9.333333   
           Phoenix Suns         Milwaukee Bucks        48.333333   7.333333   
           Toronto Raptors      Cleveland Cavaliers    56.066667   8.533333   
           Utah Jazz            Oklahoma City Thunder  62.933333   9.466667   

                                                             3pa         ft  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         30.933333  11.000000   
           New York Knicks      Los Angeles Clippers   27.066667  12.533333   
           Phoenix Suns         Milwaukee Bucks        21.466667   6.933333   
           Toronto Raptors      Cleveland Cavaliers    24.266667  11.000000   
           Utah Jazz            Oklahoma City Thunder  29.866667   9.466667   

                                                             fta       orb  \
date       visitor              home                                         
2022-03-06 New Orleans Pelicans Denver Nuggets         14.266667  8.066667   
           New York Knicks      Los Angeles Clippers   16.333333  7.866667   
           Phoenix Suns         Milwaukee Bucks        10.000000  7.266667   
           Toronto Raptors      Cleveland Cavaliers    15.666667  9.000000   
           Utah Jazz            Oklahoma City Thunder  12.466667  8.666667   

                                                             drb        trb  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         24.200000  32.266667   
           New York Knicks      Los Angeles Clippers   27.066667  34.933333   
           Phoenix Suns         Milwaukee Bucks        21.066667  28.333333   
           Toronto Raptors      Cleveland Cavaliers    23.733333  32.733333   
           Utah Jazz            Oklahoma City Thunder  25.200000  33.866667   

                                                             ast       stl  \
date       visitor              home                                         
2022-03-06 New Orleans Pelicans Denver Nuggets         17.000000  7.200000   
           New York Knicks      Los Angeles Clippers   18.400000  6.733333   
           Phoenix Suns         Milwaukee Bucks        11.533333  4.200000   
           Toronto Raptors      Cleveland Cavaliers    14.800000  4.933333   
           Utah Jazz            Oklahoma City Thunder  17.066667  6.000000   

                                                            blk       tov  \
date       visitor              home                                        
2022-03-06 New Orleans Pelicans Denver Nuggets         3.266667  8.866667   
           New York Knicks      Los Angeles Clippers   4.466667  9.933333   
           Phoenix Suns         Milwaukee Bucks        3.333333  7.200000   
           Toronto Raptors      Cleveland Cavaliers    2.466667  7.733333   
           Utah Jazz            Oklahoma City Thunder  3.733333  8.866667   

                                                              pf   pts  \
date       visitor              home                                     
2022-03-06 New Orleans Pelicans Denver Nuggets         18.000000  81.2   
    

## Correlations of Bench

In [54]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in bench_15_games:
    corr_p = pearsonr(bench_15_games['target'], bench_15_games[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print statistically significant correlations
bench_corr = corr_df[corr_df['p-value'] < .05].sort_values(['corr'], axis=0, ascending=False)
bench_corr

,corr,p-value,stat
0,1.00,0.0,target
21,0.65,0.0,3pa_opp
114,0.64,0.0,3par_opp
4,0.63,0.0,3pa
113,0.62,0.0,3par
...,...,...,...
51,-0.11,0.0,mp_std
118,-0.29,0.0,orb_perc_opp
115,-0.34,0.0,ftr
117,-0.35,0.0,orb_perc


# Comparison of Starters to Bench

In [55]:
corr_df = pd.merge(starters_corr.drop(['p-value'], axis=1), 
                   bench_corr.drop(['p-value'], axis=1),
                   left_on=['stat'], right_on=['stat'],
                   how='outer',
                   suffixes=['_starter', '_bench'])
corr_df.sort_values(['stat'], axis=0)

,corr_starter,stat,corr_bench
3,0.69,3p,0.60
6,0.66,3p_opp,0.62
10,0.44,3p_opp_std,0.38
38,0.12,3p_perc,0.04
47,0.04,3p_perc_opp,-0.02
...,...,...,...
57,-0.02,trb_perc,-0.05
55,0.02,trb_perc_opp,0.05
58,-0.02,trb_std,0.06
11,0.41,ts_perc,0.36


## Save dataframe with significantly correlated stats

In [56]:
starter_stats = starters_corr[starters_corr['corr'].abs() >= .6]['stat']
starters_df = starters_15_games[starter_stats]

bench_stats = bench_corr[bench_corr['corr'].abs() >= .6]['stat']
bench_df = bench_15_games[bench_stats]

df = pd.merge(starters_df, bench_df, 
              left_on=['date', 'visitor', 'home'], 
              right_on=['date', 'visitor', 'home'], 
              how='outer', suffixes=['_starters', '_bench'])

df = df.drop(['target_bench', 'target_starters'], axis=1)

df.columns

# df.to_csv('backend/data/inputs/3p/game_details.csv')

Index(['3pa_starters', '3par_starters', '3p_starters', '3pa_opp_starters',
       '3par_opp_starters', '3p_opp_starters', '3pa_opp_bench',
       '3par_opp_bench', '3pa_bench', '3par_bench', '3p_opp_bench',
       '3p_bench'],
      dtype='object')